# Exercise 1: Introduction to Cybersecurity Data Analysis

### Instructions

Please complete the notebook below, answering **[Questions]** and completing **[Tasks]**.  You're welcome to work with a partner, but everyone is individually responsibile for submitting their assignment.  (turning in both the .ipynb notebook and PDF document).    

Due: next **Wednesday, January 23 @ 6PM**

In [1]:
# imports, jupyter magic commands
%matplotlib inline

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import random

plt.rcParams["figure.figsize"] = (15,5)

In [2]:
# Data file
conn_log_data_path = 'data/conn.log'
# for colab, use:
# conn_log_data_path = ''

# 1. Read in Data

## File I/O
First off, let's take a look at a log file generated from Bro this log is similar to netflow logs as well. However, this log file is rather large and doesn't fit in memory.  
As part of the first exercise, figure out what setting the variable sample_percent should be in order to read in between 200k and 300k worth of (randomly selected) lines from the file. Change the variable, after doing that either click the play button above (it's the arrow) or hit the [Shift]+[Enter] keys as the same time.

In [3]:
logfile = conn_log_data_path
sample_percent = .4
num_lines = sum(1 for line in open(logfile))
slines = set(sorted(random.sample(range(num_lines), int(num_lines * sample_percent))))
print("{} lines in {}, using a sample of {} lines".format(num_lines, logfile, len(slines)))

22694356 lines in data/conn.log, using a sample of 9077742 lines


**File Creation**  
Let's write that sample to a logfile

In [8]:
outfile = 'data/conn_sample.log'
f = open(outfile, 'w+')
i = open(logfile, 'r+')

linecount = 0
for line in i:
    if linecount in slines:
        f.write(line)
    linecount += 1
f.close()
i.close()

**Reading CSVs with Pandas**
This next cell does a couple of things, first it imports pandas so we can create a dataframe, and then it reads our newly created file from above into memory. You can see the separator is specified to "\t" because Bro produces tab-delimited files by default. In this case we've also specified what we should call the columns in the dataframe.

In [9]:
df = pd.read_csv(outfile, sep="\t", header=None, nrows=100000, 
                 names=['ts','uid','id.orig_h','id.orig_p','id.resp_h','id.resp_p','proto','service','duration','orig_bytes','resp_bytes','conn_state','local_orig','missed_bytes','history','orig_pkts','orig_ip_bytes','resp_pkts','resp_ip_bytes','tunnel_parents','threat','sample'])
df.shape

(100000, 22)

In [ ]:
df = pd.read_csv(conn_log_data_path, sep="\t", header=None, nrows=100000, 
                 names=['ts','uid','id.orig_h','id.orig_p','id.resp_h','id.resp_p','proto','service','duration','orig_bytes','resp_bytes','conn_state','local_orig','missed_bytes','history','orig_pkts','orig_ip_bytes','resp_pkts','resp_ip_bytes','tunnel_parents','threat','sample'])

- pd.read_csv reads in a CSV
- nrows: the number of lines to read in

**[Task]** View DataFrame  
use .head() to show first 10 rows of DataFrame ('df')

## Section 1 Questions & Tasks
**[Question]** What is Bro Network Monitor?  
**[Question]** What is it used for?  
**[Question]** What are the different log file types and what do they describe?  What is the log file type that we have for this exercise?

# 2. Explore Data
## Pandas Data Manipulation
**[Task]** use .shape on df to show the rows, columns

**Data Summarization**  
You can see a summary of all of the numerical columns with .describe()  
**[Task]** use .describe() on df

**[Question]** What do you see?

Why doesn't the ts column look like a timestamp?  

**Data Types**  
**[Task]** use .dtypes on df

**[Question]** What kind of datatype is df['ts']?

**Converting Column Data Types**  
Time to change the ts column to a datetime object! We will accomplish that by using a simple function provided called to_datetime(). The cell below runs this function on the ts column (what should be a time stamp), and then re-assigns this column back to the dataframe in the same place. A new timestamp column could have been added to the dataframe as well so both the float value and the datetime object columns are present.  

Run the cell below to convert the column type.

In [ ]:
df['ts'] = [datetime.fromtimestamp(float(date)) for date in df['ts'].values]
df.head(1)

**[Task]** It's really handy to set the index to the timestamp.  Use .set_index() to set the dataframe index to 'ts' columns.  Use the argument, inplace=True

**[Task/Question]** Verify that the conversion was successful. What is the datatype of the column now?  

Scroll back up the page and note where you ran the describe() function. You'll see under the threat and sample columns there is likely the value of NaN. This stands for Not a Number and is a special value assigned to empty column values. There are a few ways to explore what values a column has. Two of these are value_counts() and unique().  

**[Task]** Try .value_counts() and .unique() below on different columns in the DataFrame. You can create new cells or if you want to get more than the last command worth of output you can put a print statement in front.  

**[Question]** What happens when you run them on a column with IPs (id.orig_h, id.resp_h)? What about sample or threat?  

**Remove Unnecessary Columns**  
Let's drop the threat and sample columns since they are empty.   
**[Task]** Search pandas documentation for using the .drop() method.  

**Row Selection**  
Let's find all the rows  where the value in a column match a keyword

In [ ]:
ssl = df[df['service'] == 'ssl']
ssl.head(3)

**[Question]** What type of data is in the SSL DataFrame?

We can take that subset of the dataframe, the SSL service rows, and then do further filtering...

In [ ]:
ssl_df = df[df['service'] == 'ssl']
ssl_df[ssl_df['id.resp_p'] != 443].head()

In [ ]:
http_df = df[df['service'] == 'http']
http_df[http_df['id.resp_p'] != 80]['id.orig_h'].value_counts()

## Section 2 Tasks & Questions

**[Question]** What is the line [ssl_df[ssl_df['id.resp_p'] != 443].head()] doing?

**[Task]** Show rows that use a service on a non-standard port  
Hint: first identify all of the protocols in the service columns   
**[Question]** What can this type of filtering be used for?  (Why do we care to know that services are being used on a non-standard port?  Can you identify an attack that uses this?

**[Question]** What five IP adddresses have the most traffic coming from them?

In [ ]:
df.head(1)

# 3. SSH Logs

In [ ]:
df = pd.read_csv('data/ssh.log', sep="\t", header=None, usecols=[0,1,2,3,4,5,6,7,8,9],names=['ts', 'uid', 'src_ip', 'src_port', 'server', 'server_port', 'auth_success', 'direction', 'version', 'version2'])
df.head(1)

**[Question]** How many records are there?

**[Task]** Convert Time  
**[Question]** What's the begin date and end dates of the dataset?  hint: .max()

**[Question]** What proportion of the records were successful at authenticating? hint: normalize=True

**[Task]** Did any of the records use a non-standard SSH port?